In [ ]:
# Import delle librerie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re # Modulo regex per estrarre numeri/pattern

# Stile
sns.set(style="whitegrid", context="talk")
plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
df = pd.read_csv(
    "Cars_Datasets_2025.csv",
    sep=",",
    engine="python",
    quotechar='"',
    encoding="cp1252", # Windows encoding
    on_bad_lines="warn"
)

PULIZIA DEL DATASET: le colonne contengono unità di misura e simboli -> vanno ripulite

In [ ]:
# Controllo valori unici per vedere se ci sono errori di battitura

for col in ["Company Names", "Fuel Types"]:
    print(col, df[col].unique())

In [ ]:
# --- Normalizzazione Company Names ---
df["Company Names"] = (
    df["Company Names"]
    .astype(str)
    .str.strip()
    .str.upper()
    .str.replace(r"\s+", " ", regex=True)
)

# Normalizza le stringhe
df["Fuel Types"] = (
    df["Fuel Types"]
    .astype(str)
    .str.lower()
    .str.strip()
    .str.replace(r"[(),+]", " ", regex=True) # Rimuove simboli
    .str.replace(r"[-/]", " ", regex=True) # Uniforma separatori
    .str.replace(r"\s+", " ", regex=True) # Riduce spazi multipli
)

# Funzione per classificare Fuel Types
def normalize_fuel(x):
    t = x.strip()

    # PLUG-IN HYBRID
    if re.search(r"\bplug in\b", t) or re.search(r"\bplug\b", t):
        return "plug-in hybrid"
    
    # PETROL HYBRID (qualsiasi combinazione petrol + hybrid)
    if ("petrol" in t and "hybrid" in t) or ("hybrid" in t and "petrol" in t):
        return "petrol hybrid"
    
    # HYBRID semplice (solo la parola hybrid o altre combinazioni non plug-in/petrol)
    if "hybrid" in t:
        return "hybrid"
    
    # PETROL puro
    if "petrol" in t:
        return "petrol"
    
    # DIESEL
    if "diesel" in t:
        return "diesel"
    
    # ELECTRIC / EV
    if "electric" in t or "ev" in t:
        return "electric"
    
    # HYDROGEN
    if "hydrogen" in t:
        return "hydrogen"

    return t # Se nulla matcha -> restituisco il testo normalizzato

# Applico la normalizzazione
df["Fuel Types"] = df["Fuel Types"].apply(normalize_fuel)
print("Valori finali Fuel Types:")
print(df["Fuel Types"].unique())

In [ ]:
# Controllo se ci sono duplicati

duplicates = df[df.duplicated()]
print(duplicates)

In [ ]:

def extract_numbers_from_string(s): # Estrae tutti i numeri (interi o con decimali) da una stringa

    if pd.isna(s): # Se la stringa è NaN o non-text, ritorna []
        return []
    
    # Forzo a stringa (in caso di numeri già presenti)
    s = str(s)
    
    # Trova tutte le occorrenze numeriche (es: "70-85" -> ["70","85"], "300 (est.)" -> ["300"])
    nums = re.findall(r'\d+\.?\d*', s)
    return nums

In [ ]:
def parse_numeric_value(s, prefer_mean_for_range=True):
    
    """
        Normalizza un campo testuale che contiene:
        - un singolo numero -> restituisce quel numero (float)
        - un range 'low-high' -> restituisce la media (float) se prefer_mean_for_range True, altrimenti restituisce il low
        - valori con testo tra parentesi '300 (est.)' -> estrae 300
        - valori con '+', ',' simboli -> pulisce e interpreta il numero
        
        Se non trova numeri ritorna nan.
    """
    
    nums = extract_numbers_from_string(s)
    
    if not nums: # Nessun numero -> nan
        return np.nan
    
    if len(nums) == 1: # Singolo numero -> ritorna float
        return float(nums[0])
    
    else: # più numeri -> interpreto come range -> media
        arr = [float(x) for x in nums]
        if prefer_mean_for_range:
            return float(np.mean(arr))
        else:
            return float(arr[0])  # o arr[0] come low

In [ ]:
def clean_price_generic(price_str):
    """
        Versione per i prezzi:
        - rimuove simboli $ e spazi e virgole
        - se c'è un range, usa la media
        - se non trova numeri ritorna NaN
    """
    
    if pd.isna(price_str):
        return np.nan
    
    s = str(price_str).replace("$", "").replace(",", "").strip()
    
    return parse_numeric_value(s, prefer_mean_for_range=True) # usa parse_numeric_value per estrarre numero o media se range

In [ ]:
# ---- HorsePower: gestisco range come media, rimuovo 'hp' se presente ----
df["HorsePower_clean"] = df["HorsePower"].apply(parse_numeric_value)

# ---- Total Speed: gestisco '300 (est.)', range, ecc. ----
df["TotalSpeed_clean"] = df["Total Speed"].apply(parse_numeric_value)

# ---- Performance (0-100) in secondi: ex. '2.5 sec' o '10.5 sec' ----
df["Performance_0_100_clean"] = df["Performance(0 - 100 )KM/H"].apply(parse_numeric_value)

# ---- CC/Battery Capacity: rimuovo 'cc', virgole, ecc. ----
df["CC_clean"] = df["CC/Battery Capacity"].apply(parse_numeric_value)

# ---- Seats: può essere già numerico ma uso pd.to_numeric con coercion -> converte valori non interpretabili in nan invece di lanciare ValueError ----
df["Seats_clean"] = pd.to_numeric(df["Seats"], errors="coerce")

# ---- Torque: potrebbe contenere '100 - 140 Nm' oppure '900 Nm' ----
df["Torque_clean"] = df["Torque"].apply(parse_numeric_value)

# ---- Cars Prices: usa funzione che pulisce $ , e gestisce range ----
df["CarsPrices_clean"] = df["Cars Prices"].apply(clean_price_generic)

CONTO VALORI NULLI

In [ ]:
# Contare i nulli per ogni colonna
print("\n--- CONTEGGIO VALORI NULLI PRE-PULIZIA ---")
null_counts = df.isnull().sum()
print(null_counts)

# Visualizzazione grafica dei dati mancanti
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=False, yticklabels=False, cmap='viridis')

plt.title('Mappa dei Valori Mancanti (Giallo = Null)')
plt.show()

In [ ]:
# Elimino le righe che contengono nulli ovunque
righe_con_null = df[df.isnull().any(axis=1)]
print("Righe eliminate:")
display(righe_con_null)

df = df.dropna()

In [ ]:
print("\n--- CONTEGGIO VALORI NULLI POST-PULIZIA ---")
null_counts = df.isnull().sum()
print(null_counts)

# Visualizzazione grafica dei dati mancanti
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=False, yticklabels=False, cmap='viridis')

plt.title('Mappa dei Valori Mancanti (Giallo = Null)')
plt.show()

INFO E STATISTICHE DESCRITTIVE

In [ ]:
# Info dataset
print("\n--- INFO GENERALI ---")
df.info()

In [ ]:
# Statistiche descrittive
print("\n--- DESCRIZIONE STATISTICA ---")
print(df.describe())

OUTLIER

In [ ]:
numeric_cols = [
    "HorsePower_clean",
    "TotalSpeed_clean",
    "CarsPrices_clean",
    "Torque_clean",
    "CC_clean",
    "Performance_0_100_clean"
]

In [ ]:
def find_outliers(df, col): # Funzione per calcolare IQR e individuare outlier
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    return outliers

In [ ]:
for col in numeric_cols: # Loop su tutte le colonne
    plt.figure(figsize=(8,4))
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot di {col} (outlier evidenziati)')
    plt.show()

    outliers = find_outliers(df, col)
    print(f"\nColonna: {col}")
    print(f"Numero di outlier: {len(outliers)}")
    if len(outliers) > 0:
        print(outliers[[col]])

MATRICE CORRELAZIONE

In [ ]:
# Matrice di correlazione tra colonne numeriche
corr = df.corr(numeric_only=True)

# Rappresentazione con heatmap
sns.heatmap(
    corr, 
    annot=True, # Mostra i valori numerici
    cmap="coolwarm"
)

plt.title("Correlation Matrix")
plt.show()

RELAZIONE: CAVALLI VS PERFORMANCE

In [ ]:
# Grafico a dispersione
sns.scatterplot(
    data=df, 
    x="HorsePower_clean", 
    y="Performance_0_100_clean",
    s=200   # Dimensione punti
)

plt.title("HorsePower vs 0–100 Performance")
plt.xlabel("Horse Power (hp)")
plt.ylabel("0–100 (sec)")
plt.show()

RELAZIONE: CAVALLI VS TOTAL SPEED

In [ ]:
# Grafico a dispersione
sns.scatterplot(
    data=df, 
    x="HorsePower_clean", 
    y="TotalSpeed_clean",
    s=200   # Dimensione punti
)

plt.title("HorsePower vs Total Speed")
plt.xlabel("Horse Power (hp)")
plt.ylabel("Total Speed (km/h)")
plt.show()